In [1]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import pyodbc
from sklearn.feature_extraction.text import TfidfVectorizer 

In [ ]:
nltk.download('punkt')

In [ ]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [ ]:
articlesContent = execQuery("""Select content
from article""")

In [2]:
#connection = psycopg2.connect(user = "postgres",
#                                      password = "root",
#                                      host = "localhost",
#                                      port = "5432",
#                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random) 
filepath = '/home/daniel/OneDrive/KUuni/DataScience/Python/small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)

In [3]:
df_content = df['content'].str.lower()
series_content_tokens = df_content.apply(nltk.word_tokenize)

In [4]:
series_content_tokens = series_content_tokens.explode()
df_content_tokens = series_content_tokens.to_frame()
df_content_tokens = df_content_tokens.reset_index().rename(columns={'index':'content_index'})
series_content_tokens = df_content_tokens.groupby('content_index')['content'].value_counts(normalize=True)

In [5]:
series_content_tokens

content_index  content     
0              the             0.084715
               .               0.040516
               of              0.036832
               to              0.029466
               in              0.025783
                                 ...   
249            unacceptable    0.007752
               wa              0.007752
               wednesday       0.007752
               will            0.007752
               yangon          0.007752
Name: content, Length: 76994, dtype: float64

In [19]:
# create TF
TF = series_content_tokens.unstack(level=1)
TF

content,!,#,$,%,&,','','come,'d,'editing,...,…,…..,…and,…liberate,…record,…researchers,€1100,€7.2bn,→,≥8
content_index,,,,,,,,,,,,,,,,,,,,,
0,0.007366,NaN,0.001842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.008969,NaN,NaN,0.004484,0.004484,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.001912,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.001922,NaN,0.001281,0.004484,0.000641,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,0.002985,NaN,NaN,NaN,0.002985,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,NaN,0.033708,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [341]:
import numpy as np
import nltk

df_1 = pd.DataFrame({'sent': [
                     'the sky is blue',
                     'blue not is sky the']})

series_1 = df_1['sent'].str.split()

# used to to tokenize
    # # Tokenize and generate count vectors
    # word_vec = series_of_lists_of_tokens.str.split().apply(pd.value_counts).fillna(0)


def tf_idf(series_of_lists_of_tokens):
    """ Returns tf-idf matrix(dataframe) for a given df-column/series with list of tokens for each row. """

    # Generate count vectors
    word_vec = series_of_lists_of_tokens.apply(pd.value_counts).fillna(0) # intuitive but slow


    # Compute term frequencies
        ### Assuming from http://www.tfidf.com/ -> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
    tf = word_vec.divide(np.sum(word_vec, axis='columns'), axis='index')

    # Compute inverse document frequencies
        ### Slides seems wrong s.18: https://docs.google.com/presentation/d/1rXhewxbmLX7mzDcy8aucI8OLIikfrLLiz8ocN5GS06I/edit#slide=id.g77508fe5d3_0_19
        ### Assuming from http://www.tfidf.com/ -> log_e(Total number of documents / Number of documents with term t in it)
    idf = np.log(len(tf) / word_vec[word_vec > 0].count()) 

    # Compute TF-IDF vectors and return
    tfidf = np.multiply(tf, idf.to_frame().T)
    
    return tfidf

# print(tf)
# print(idf)
# stemming = PorterStemmer()
series_content_tokens = df_1['sent'].str.split()

# def tokenize(text):
#     list_tokens = nltk.word_tokenize(text)
#     a = [stemming.stem(word) for word in list_tokens]
#     return a

# df_tokens = series_content_tokens.explode().to_frame().reset_index().rename(columns={'index':'content_index'})
# df_tokens.groupby('content_index')[df_tokens.columns[1]].value_counts(normalize=True).unstack(level=1)


tf_idf(series_content_tokens)
# df_tokens.apply(tuple).value_counts()
# df_1['sent'].str.split().groupby('name')['sent'].value_counts().unstack().fillna(0)

pd.DataFrame(series_content_tokens.tolist())

0.0007521289603870707

In [388]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df['sent'])

In [1]:
feature_names = tfidf.get_feature_names()
corpus_index = [n for n in corpus]
import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
print(df)

NameError: name 'tfidf' is not defined

In [111]:
import numpy as np

df_1 = pd.DataFrame({'sent': 
                    ['This This is is first', 
                     'This is the second sentence',
                     'This the third sentence']})

# Tokenize and generate count vectors
word_vec = df_1.sent.str.split().apply(pd.value_counts).fillna(0)

# Compute term frequencies
tf = word_vec.divide(np.sum(word_vec, axis='columns'), axis='index')

# Compute inverse document frequencies
idf = np.log(len(tf) / word_vec[word_vec > 0].count()) 

# Compute TF-IDF vectors
tfidf = np.multiply(tf, idf.to_frame().T)

print(word_vec[word_vec > 0].count())

This        3
is          2
first       1
second      1
the         2
sentence    2
third       1
dtype: int64


In [70]:
df = pd.DataFrame({'docId': [1,2,3], 
               'sent': ['This is the first sentence', 
                        'This is the second sentence',
                        'This is the third sentence']})

# use 'value_counts' to get counts of items in list
tf = df.sent.apply(pd.value_counts).fillna(0) 
# tf = df.sent.str.split().apply(pd.value_counts).fillna(0)

# add one to numerator and denominator just incase a term isn't in any document
# maximum value is log(N) and minimum value is zero
tf

,This is the first sentence,This is the second sentence,This is the third sentence
0,26.0,0.0,0.0
1,0.0,27.0,0.0
2,0.0,0.0,26.0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
df = pd.DataFrame({'docId': [1,2,3], 
               'sent': ['This is the first sentence','This is the second sentence', 'This is the third sentence']})

v = TfidfVectorizer()
x = v.fit_transform(df['sent'])
 
type(x)

scipy.sparse.csr.csr_matrix